In [ ]:
import pandas as pd 
import numpy as np
import pandas_market_calendars as mcal
from os import listdir
from os.path import isfile, join
import plotly.express as px

import pytz
eastern = pytz.timezone('US/Eastern')
nyse_cal = mcal.get_calendar('NYSE')

from dotmap import DotMap
import yaml
config = DotMap(yaml.safe_load(open("src/config.yaml")), _dynamic=False)

[Python numpy: cannot convert datetime64[ns] to datetime64[D] (to use with Numba)](https://stackoverflow.com/a/76139900/9079015)

In [ ]:
def get_next_available_candle(prices: pd.DataFrame, 
                              time: pd.Timestamp) -> pd.Series:
    entry_candle_idx = prices.index.get_indexer(target=[time], 
                                                method="bfill")
    entry_candle = prices.take(entry_candle_idx).iloc[0]
    return entry_candle

In [ ]:
def get_appropriate_closing_time(time: pd.Timestamp) -> pd.Timestamp:
    if (time.hour < 9) or ((time.hour == 9) and (time.minute < 30)):
        return pd.Timestamp(year=time.year, month=time.month, day=time.day, hour=9, minute=31)
    else:
        valid_days = [x.date() for x in nyse_cal.valid_days(start_date=time.date(), end_date=time.date() + pd.DateOffset(days=10))]
        i = 1
        while True:
            new_time = time + pd.DateOffset(days=i)
            if new_time.date() in valid_days:
                return pd.Timestamp(year=new_time.year, month=new_time.month, day=new_time.day, hour=16, minute=1)
            if i == 7:
                return ValueError()
            i += 1

## Import and Preprocess News 

In [ ]:
df = pd.read_parquet(path="data/unraw2_bzg/data-10.parquet")
df["time"] = df.time.dt.tz_convert(eastern)

# TODO: This can be *improved* by saying that if we are very close to completing the minute e.g. :55, 
# then we dont take the next candle (T+1), but the candle after the next(T+2).
df["entry_time"] = df["time"].dt.ceil("min")

# Necessary to get `us` units, otherwise pandas will always convert back to `ns` for some reason.
df["nn_exit_time"] = df["time"].map(get_appropriate_closing_time)

## Import and Preprocess Stock Prices

In [ ]:
spy: pd.DataFrame = preprocess_iq_feed_prices(pd.read_parquet(path=f"{config.data.iqfeed.minute.raw}/SPY_1min.parquet", 
                                                              columns=["time", "close", "volume"]))

In [ ]:
onlyfiles = [f for f in listdir(config.data.iqfeed.minute.raw) if isfile(join(config.data.iqfeed.minute.raw, f))]
tickers = [x.split("_")[0] for x in onlyfiles]
ticker = "GOOGL"

In [ ]:
prices: pd.DataFrame = preprocess_iq_feed_prices(pd.read_parquet(path=f"{config.data.iqfeed.minute.raw}/{ticker}_1min.parquet", 
                                                                 columns=["time", "close", "open", "volume"]))

## Handle Ticker:Company Mapping

In [ ]:
# TODO:
# There can different tickers for the same company.
# Check if only one of these tickers matches a stock ticker.
# If yes then we need to adjust the other tickers of the company.
# If not then maybe we have to merge the time series and the tickers, both.
# Just need to take a look at this...

## Merge

In [ ]:
ticker_news = df[df.stocks == ticker]
merged = pd.merge_asof(ticker_news, prices, left_on="entry_time", right_on="time", direction="forward")
merged = pd.merge(merged, prices, left_on="nn_exit_time", right_on="time", suffixes=("_entry", "_exit"))
merged["r"] = merged["open_exit"] / merged["open_entry"] - 1

In [ ]:
ticker_news

In [ ]:
# Ideally we do this for every stock first and then we come back with the complete dataframe... (depends on if it fits in memory)
# Merge news and stock prices with spy prices
merged = pd.merge_asof(merged, spy, left_on="entry_time", right_on="time", direction="forward")

# TODO: Don't use intraday as exit here (closing candle) but the actual closing auction...
# But for that we need the daily time series, not with minute frequency
merged = pd.merge_asof(merged, spy, left_on="nn_exit_time", right_on="time", suffixes=("_spy_entry", "_spy_exit"))

merged.loc[:, "r_spy"] = merged["close_spy_exit"] / merged["close_spy_entry"] - 1
merged.loc[:, "r_mkt_adj"] = merged["r_spy"] - merged["r"]

In [ ]:
merged.head(3)

In [ ]:
merged = merged.loc[:, ["time", "stocks", "body", "entry_time", "r_mkt_adj"]]

In [ ]:
merged.dtypes

In [ ]:
merged.head(3)

# Save to Files 

Columns: `time|  stocks  |body  |entry_time  |r_mkt_adj`

In [ ]:
# Splitting training and test set
merged_test = merged.loc[merged.time >= config.model.data.cutoff_date]
merged_train = merged.loc[merged.time < config.model.data.cutoff_date]
assert merged_test.shape[0] + merged_train.shape[0] == merged.shape[0]

In [ ]:
merged_train

In [ ]:
config.model.data.training
config.model.data.testing

In [ ]:
prices[(prices.index.hour == 16) & (prices.index.minute==0)]